# Churn Prediction 

Dataset: https://www.kaggle.com/datasets/safrin03/predictive-analytics-for-customer-churn-dataset/data

About Dataset
Context :

This dataset is part of a data science project focused on customer churn prediction for a subscription-based service. Customer churn, the rate at which customers cancel their subscriptions, is a vital metric for businesses offering subscription services. Predictive analytics techniques are employed to anticipate which customers are likely to churn, enabling companies to take proactive measures for customer retention.

Content :

This dataset contains anonymized information about customer subscriptions and their interaction with the service. The data includes various features such as subscription type, payment method, viewing preferences, customer support interactions, and other relevant attributes. It consists of three files such as "test.csv", "train.csv", "data_descriptions.csv".

Columns :

- *CustomerID*: Unique identifier for each customer
- *SubscriptionType*: Type of subscription plan chosen by the customer (e.g., Basic, Premium, Deluxe)
- *PaymentMethod*: Method used for payment (e.g., Credit Card, Electronic Check, PayPal)
- *PaperlessBilling*: Whether the customer uses paperless billing (Yes/No)
- *ContentType*: Type of content accessed by the customer (e.g., Movies, TV Shows, Documentaries)
- *MultiDeviceAccess*: Whether the customer has access on multiple devices (Yes/No)
- *DeviceRegistered*: Device registered by the customer (e.g., Smartphone, Smart TV, Laptop)
- *GenrePreference*: Genre preference of the customer (e.g., Action, Drama, Comedy)
- *Gender*: Gender of the customer (Male/Female)
- *ParentalControl*: Whether parental control is enabled (Yes/No)
- *SubtitlesEnabled*: Whether subtitles are enabled (Yes/No)
- *AccountAge*: Age of the customer's subscription account (in months)
- *MonthlyCharges*: Monthly subscription charges
- *TotalCharges*: Total charges incurred by the customer
- *ViewingHoursPerWeek*: Average number of viewing hours per week
- *SupportTicketsPerMonth*: Number of customer support tickets raised per month
- *AverageViewingDuration*: Average duration of each viewing session
- *ContentDownloadsPerMonth*: Number of content downloads per month
- *UserRating*: Customer satisfaction rating (1 to 5)
- *WatchlistSize*: Size of the customer's content watchlist

Acknowledgments :
The dataset used in this project is obtained from Data Science Challenge on Coursera and is used for educational and research purposes. Any resemblance to real persons or entities is purely coincidental.


**The problem definition:**

The goal of this project is to build a predictive model that identifies customers who are likely to churn (i.e., stop using the service). Using historical customer data — including demographics, subscription details, and usage behavior — we aim to classify whether a customer will churn or remain active.


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn_pandas import DataFrameMapper

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBRFClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer,  recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split


from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

from datetime import datetime, timedelta


In [ ]:
df_test=pd.read_csv('test.csv')
df_train=pd.read_csv('train.csv')

# EDA and preprocessing

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.columns

In [ ]:
df_test.columns

In [ ]:
(df_train.shape, df_test.shape)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
# there are no missing values
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
# check class balance
df_train.groupby('Churn').CustomerID.nunique()

Classes are unbalanced. 

The strategy of metrics: Do not use accuracy, but F1-score (only for Class 1), ROC-AUC

Balancing classes: Undersampling (delete the "zero" part of the users), oversampling (duplicate "units" or use SMOTE), or use models that take into account class weights (ex LogisticRegression)

In [ ]:
numeric_col=[]
for col  in df_train.columns:
    if(df_train[col].dtypes in ('float64','int64')):
        numeric_col.append(col)
print(f"Count of numeric columns in the datset - {len(numeric_col)}")
print(numeric_col)

numeric_col.remove('Churn')

In [ ]:
#Find categorical  Columns
categorial_col=[]
for col  in df_train.columns:
    if(df_train[col].dtypes=='object'):
        categorial_col.append(col)
print(f"Count of categorical columns in the datset - {len(categorial_col)}")
print(categorial_col)

In [ ]:
del df_train['CustomerID']
del df_test['CustomerID']

### Numeric Features Vizualization

In [ ]:
def plot_feature_distribution(df, feature: str, title_feature: str = None, nbins: int = 60):

    if title_feature is None:
        title_feature = feature

    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(f"Distribution of {title_feature} by Churn", 
                        f"Boxplot: {title_feature} by Churn"),
        shared_xaxes=False
    )

    hist_data = px.histogram(
        df,
        x=feature,
        color="Churn",
        barmode="stack",
        nbins=nbins,
        opacity=0.7
    )

    for trace in hist_data.data:
        trace.showlegend = True
        fig.add_trace(trace, row=1, col=1)

    box_data = px.box(
        df,
        x="Churn",
        y=feature,
        color="Churn"
    )

    for trace in box_data.data:
        trace.showlegend = False
        fig.add_trace(trace, row=2, col=1)

    fig.update_layout(
        height=800,
        showlegend=True,
        legend_title="Churn",
        xaxis=dict(title=title_feature),
        xaxis2=dict(title="Churn"),
        yaxis=dict(title="Count Clients"),
        yaxis2=dict(title=title_feature),
        title_text=f"{title_feature} vs Churn — Histogram & Boxplot"
    )

    fig.show()

In [ ]:
for col in numeric_col:
    print(col)
    print(df_train.groupby('Churn')[col].describe())
    plot_feature_distribution(df_train, feature=col, title_feature=col)

So, we clearly see that: 
- Churned Users tend to have lover values for  Account Age, Total Charges, Content Downloads Per Month, Average Viewing Duration and Viewing Hours Per Week. These features are likely interrelated and potentially collinear.
- Churned users also submit more support tickets, which may indicate a higher level of dissatisfaction, confusion, or unresolved technical issues — possibly contributing to their decision to leave.
- Two features stand out as non-intuitive:
    - Monthly Charges are higher among churned users, which might suggest pricing sensitivity or billing-related friction. This relationship needs further analysis
    - Watchlist Size is also higher among churned users. A possible interpretation is that these users are interested in content but face e.g., technical issues, save items for later rather than watch them. A next step would be to explore the correlation between Watchlist Size and actual Viewing Time.




### Categorial Features Visualization 

In [ ]:

def plot_pie_by_churn(df, category_col: str, title: str = None):

    if title is None:
        title = f"{category_col} Distribution by Churn"

    categories = df[category_col].dropna().unique().tolist()

    default_colors = px.colors.qualitative.Plotly
    color_map = {
        cat: default_colors[i % len(default_colors)]
        for i, cat in enumerate(categories)
    }

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'domain'}, {'type': 'domain'}]],
        subplot_titles=[
            f"{category_col} (Churn = 0)",
            f"{category_col} (Churn = 1)"
        ]
    )

    pie_0 = px.pie(
        df[df['Churn'] == 0],
        names=category_col,
        hole=0.33,
        category_orders={category_col: categories},
        color=category_col,
        color_discrete_map=color_map
    )

    pie_1 = px.pie(
        df[df['Churn'] == 1],
        names=category_col,
        hole=0.33,
        category_orders={category_col: categories},
        color=category_col,
        color_discrete_map=color_map
    )

    for trace in pie_0.data:
        fig.add_trace(trace, row=1, col=1)

    for trace in pie_1.data:
        fig.add_trace(trace, row=1, col=2)

    fig.update_layout(
        title_text=title,
        legend_title=category_col,
        showlegend=True,
        height=500
    )

    fig.show()

In [ ]:
categorial_col=categorial_col[:-1]

In [ ]:
for col in categorial_col:
    plot_pie_by_churn(df_train, category_col=col, title=f"{col} by Churn Statuses")

There are fewer clear distinctions in the categorical features between churned and non-churned users. However, a few patterns can be tentatively noted:

- Churned users are more likely to have a Basic subscription, while non-churned users tend to subscribe to Premium plans, which may indicate a link between churn and perceived value or feature limitations.

- There are also minor differences in the distribution of Payment Methods, but these patterns are less pronounced and would require statistical testing to validate.

In [ ]:
numeric_df = df_train.select_dtypes(include="number")
corr = numeric_df.corr(numeric_only=True)

plt.figure(figsize=(10, 6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", center=0, square=True)
plt.title("Correlation Matrix of Numeric Features")
plt.tight_layout()
plt.show()

In [ ]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

features = [
    'AccountAge', 'MonthlyCharges', 'TotalCharges',
    'ViewingHoursPerWeek', 'AverageViewingDuration',
    'ContentDownloadsPerMonth', 'UserRating',
    'SupportTicketsPerMonth', 'WatchlistSize'
]

X = df_train[features].copy()

X_scaled = StandardScaler().fit_transform(X)

vif_df = pd.DataFrame()
vif_df['feature'] = features
vif_df['VIF'] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

print(vif_df)

We see a high colinearity between AccountAge and TotalCharges. For linear models, we will remove the TotalCharges feature to avoid multicollinearity (this worsens the stability of the coefficients and confuses interpretation). Why TotalCharges? Usually, monetary metrics depend on many factors and are more difficult to interpret. For tree models (Decision Tree, Random Forest, XGBoost), you can leave both features. These models do not suffer from multicollenarity, they choose the desired feature themselves.

## Extra: one look at outlier detection methods

Let's take a short look at the methods of detecting outliers 

###  Boxplot / Visual Inspection

What it is - Visual method based on the IQR rule (Interquartile Range). Outliers are values outside the whiskers.

Pros:
- Quick and intuitive
- Highlights extreme values clearly

Cons:
- Only works for 1D (single feature at a time)
- Not scalable for high dimensions

In [ ]:
X.plot(kind="box", subplots=True, figsize=(12, 12), layout=(3, 3))
plt.show()

###  IQR 

- defines outliers as values beyond 1.5×IQR from Q1/Q3.

Pros:
- Very easy to implement
- No assumptions about distribution

Cons: 
- Ignores relationships between features
- Not useful for multivariate anomalies


In [ ]:
def count_iqr_outliers(df, threshold=1.5):

    numeric_cols = df[features]
    outlier_counts = {}

    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower = Q1 - threshold * IQR
        upper = Q3 + threshold * IQR

        outliers = (df[col] < lower) | (df[col] > upper)
        outlier_counts[col] = outliers.sum()

    return pd.Series(outlier_counts).sort_values(ascending=False)

In [ ]:
outliers_dict = count_iqr_outliers(df_train)
outliers_dict

### LocalOutlierFactor (LOF)

Density-based algorithm. Compares local density of a point to its neighbors.

Pros:
- Captures local anomalies
- No need to label data (unsupervised)

Cons:

- Sensitive to n_neighbors parameter
- It learns and evaluates only those points that were immediately given to it.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor

In [ ]:
lof = LocalOutlierFactor(n_neighbors=20)

lof.fit_predict(df_train[numeric_col])

In [ ]:
df_scores = lof.negative_outlier_factor_
print(np.sort(df_scores)[0:6])

scores = pd.DataFrame(np.sort(df_scores))
plt.figure(figsize=(14, 10))
scores.plot(stacked=True, xlim=[0, 16], style=".-", color="red")
plt.title("LocalOutlierFactor", fontsize=24)
plt.xlabel("Number of Neighbors", fontsize=12)
plt.ylabel("Scores", fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid()
plt.legend(["LOF"])
plt.show()

The graph helps you understand which number of neighbors is optimal - where the graph is already fairly stable (5-10)

### Isolation Forest 

Tree-based algorithm that "isolates" anomalies quickly by random splits.

Pros:
- Scalable and fast
- Works with mixed data
- Can be used on new/unseen data

Cons:
- Can be unstable with small datasets
- Randomness requires tuning / multiple runs

In [ ]:
from sklearn.ensemble import IsolationForest

iso = IsolationForest(contamination=0.05, random_state=42)
df_train["iso_score"] = iso.fit_predict(df_train[features])

In [ ]:
outliers = df_train[df_train["iso_score"] == -1]
inliers = df_train[df_train["iso_score"] == 1]

for col in numeric_col:
    print(f"{col}: outliers mean={outliers[col].mean():.2f}, inliers mean={inliers[col].mean():.2f}")

In [ ]:
del df_train['iso_score']

# Feature Engineering

In [ ]:
X_train= df_train.drop(columns='Churn')
y_train=df_train['Churn']

X_test = df_test 

We chose OneHotEncoder because it does not impose any artificial ordering between categories. Unlike LabelEncoder, which converts categories into integers (e.g., "Basic" → 0, "Standard" → 1, "Premium" → 2), OneHotEncoder treats each category as an independent binary feature.
This avoids misleading the model into assuming a ranking where none exists — which is especially important for nominal variables such as subscription type, payment method, or content genre.

We selected StandardScaler because it standardizes features by removing the mean and scaling to unit variance, which is optimal for most linear models, SVMs, and PCA.
Compared to MinMaxScaler, StandardScaler is less sensitive to the presence of extreme values (though not fully robust) and does not squash data into a fixed 0–1 range, preserving variance.
We did not use RobustScaler because our numerical features do not contain significant outliers that would distort mean-based scaling.

In [ ]:
binary_cols = ['PaperlessBilling', 'MultiDeviceAccess', 'ParentalControl', 'SubtitlesEnabled']

for col in binary_cols:
    if X_train[col].dtype == 'object':
        X_train[col] = X_train[col].map({'Yes': 1, 'No': 0})
        X_test[col] = X_test[col].map({'Yes': 1, 'No': 0})

X_train['Gender'] = X_train['Gender'].map({'Female': 1, 'Male': 0})
X_test['Gender'] = X_test['Gender'].map({'Female': 1, 'Male': 0})


scaled_features = [
    'AccountAge', 'MonthlyCharges', 'TotalCharges', 
    'ViewingHoursPerWeek', 'AverageViewingDuration', 
    'ContentDownloadsPerMonth', 'UserRating',
    'SupportTicketsPerMonth', 'WatchlistSize'
]

one_hot_encoding_cols = ['SubscriptionType', 'PaymentMethod', 'ContentType', 'DeviceRegistered', 'GenrePreference']


all_cols = X_train.columns.tolist()

other_features = [col for col in all_cols if col not in scaled_features + one_hot_encoding_cols]

mapper = DataFrameMapper(

    [([col], [SimpleImputer(strategy='mean'), StandardScaler()]) for col in scaled_features] +
    [([col], [SimpleImputer(strategy='most_frequent'), OneHotEncoder(drop='first', sparse_output=False)]) for col in one_hot_encoding_cols] +
    [([col], None) for col in other_features],

    input_df=True, df_out=True
)

X_train_final = mapper.fit_transform(X_train)
X_test_final = mapper.transform(X_test)


# Models 

In [ ]:
models = [
    ('LogReg', LogisticRegression(max_iter=1000)),
    ('KNN', KNeighborsClassifier()),
    ('DTClass', DecisionTreeClassifier()),
    ('RFClass', RandomForestClassifier()),
    ('GBM', GradientBoostingClassifier()),
    ("XGBoost", XGBRFClassifier()),
    ("LightGBM", LGBMClassifier())
]

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

for name, model in models:
    acc = cross_validate(model, X_train_final, y_train, cv=cv, scoring=scoring)

    print(f"Model: {name}")
    print("Recall:", round(acc["test_recall"].mean(), 4))
    print("F1:", round(acc["test_f1"].mean(), 4))
    print("ROC AUC:", round(acc["test_roc_auc"].mean(), 4))
    print(f"\n")


If the goal is to catch as many CHURN (1s) as possible (high Recall):
Best result: DecisionTree (DT) — Recall = 0.30 -> but at the same time, the terrible AUC model stupidly "predicts 1 for everyone in a row."

If you need a quality balance (F1 + AUC): 

- LogisticRegression is the most balanced: F1 = 0.1961, ROC AUC = 0.75

- LightGBM and GBM are close in AUC, but Recall is lower.

## 1 - SMOTE

SMOTE (Synthetic Minority Oversampling Technique) is a smart way to "add" more than 1 without copying them, but creating new similar ones.
It is used when it is necessary to improve recall or F1 in a minority class, especially when there is an imbalance.

In [ ]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_final, y_train)

In [ ]:
smote = SMOTE(random_state=42)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
            }

models = [
    ("LogReg", LogisticRegression(class_weight=None, max_iter=1000, random_state=42)),
    ("RandomForest", RandomForestClassifier(class_weight=None, n_estimators=100, random_state=42)),
    ("DecisionTree", DecisionTreeClassifier(class_weight=None, random_state=42))
]


for name, model in models:
    pipe = Pipeline([
        ('smote', smote),
        ('clf', model)
    ])
    
    acc = cross_validate(pipe, X_train_final, y_train, cv=cv, scoring=scoring)

    print(f"Model: {name}")
    print("Recall:", round(acc["test_recall"].mean(), 4))
    print("F1:", round(acc["test_f1"].mean(), 4))
    print("ROC AUC:", round(acc["test_roc_auc"].mean(), 4))
    print(f"\n")


## 2 - class_weight='balanced'

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

models = [
    ("LogReg_balanced", LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)),
    ("RandomForest_balanced", RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)),
     ("DecisionTree_balanced", DecisionTreeClassifier(class_weight='balanced', random_state=42))

]

for name, model in models:
    acc = cross_validate(model, X_train_final, y_train, cv=cv, scoring=scoring)

    print(f"Model: {name}")
    print("Recall:", round(acc["test_recall"].mean(), 4))
    print("F1:", round(acc["test_f1"].mean(), 4))
    print("ROC AUC:", round(acc["test_roc_auc"].mean(), 4))
    print(f"\n")


In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train_final, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [ ]:
model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
model.fit(X_tr, y_tr)


y_val_pred = model.predict(X_val)
y_val_proba = model.predict_proba(X_val)[:, 1]

In [ ]:
print("Classification Report:")
print(classification_report(y_val, y_val_pred, digits=4))


print("ROC AUC:", round(roc_auc_score(y_val, y_val_proba), 4))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))

In [ ]:
threshold = 0.3
y_val_pred_custom = (y_val_proba >= threshold).astype(int)

print("Classification Report (threshold = 0.3):")
print(classification_report(y_val, y_val_pred_custom, digits=4))

print("ROC AUC:", round(roc_auc_score(y_val, y_val_pred_custom), 4))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred_custom))

We experimented with lowering the classification threshold to 0.3 in order to capture more positive cases (churned customers). While this significantly increased recall (up to ~92%), it also caused a sharp drop in precision (down to ~24%) and overall accuracy.

This means that although we correctly identified almost all churners, the model also falsely labeled a very large number of non-churners as churners. In practical terms, this could lead to wasting resources on users who are not actually at risk, which may not be sustainable in a real-world scenario.

Therefore, a threshold of 0.3 does not provide a good balance between recall and precision, and we recommend sticking with the default threshold of 0.5 or finding an optimized value based on business constraints.

The best result:
- LogisticRegression
- class_weight='balanced' (threshold = 0.5) 
- F1 = 0.1961, ROC-AUC = 0.75

## Is Logistic Regression Suitable for Predicting Churn?

Technically, yes. Logistic regression is a valid choice for predicting whether a customer will churn (binary outcome: 1 or 0). However, in our case, we have more than just the churn indicator — we also know how long a customer stays before leaving.

This means we are dealing with two variables:

- A binary variable: whether the customer churned (1/0),
- A continuous variable: the time until churn.

Generalized Linear Models (GLM)
GLM is a general modeling framework defined by the equation:

$$
g(\mathbb{E}[Y]) = \beta_0 + \beta_1 x_1 + \dots + \beta_p x_p
$$

Here:

- Y  - the target variable,
- 𝑔 - the link function,
- 𝛽 - are the model coefficients.

Logistic regression is just a special case of GLM, where the link function is the logit, and the target variable 

Time-to-Event Modeling and the Weibull Distribution
When we model time until an event (like churn), we enter the domain of survival regression. A common choice for modeling such durations is the Weibull distribution.

The Weibull distribution is flexible and defined by two parameters:

- 𝛼 (scale) — controls the spread of the distribution,
- 𝜌 (shape) — controls the hazard behavior over time.

Different values of 𝜌 allow the distribution to model increasing, decreasing, or constant risk over time.

In [ ]:
from scipy.stats import weibull_min

x_weibull = np.linspace(0, 5, 500)
shapes = [0.5, 1, 1.5, 2, 5]
plt.figure(figsize=(6, 4))
for shape in shapes:
    y_weibull = weibull_min.pdf(x_weibull, c=shape)
    plt.plot(x_weibull, y_weibull, label=f"rho = {shape}")

plt.title("Weibull Distributions with Different Shape Parameters")
plt.xlabel("Time")
plt.ylabel("Probability Density")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import lifelines as lf

In [ ]:
churn = df_train.loc[:, ['Churn', 'AccountAge', 'SubtitlesEnabled', 'ParentalControl',
                          'MonthlyCharges', 'PaperlessBilling',
                          'PaymentMethod','SubscriptionType',
                         'ViewingHoursPerWeek','UserRating',
                         'SupportTicketsPerMonth',
                         'ContentType', 'MultiDeviceAccess','DeviceRegistered',
                         'AverageViewingDuration', 'ContentDownloadsPerMonth',
                         'GenrePreference','WatchlistSize'
                ]]

In [ ]:
sns.displot(churn.AccountAge, kde = False)
plt.xlabel('Number of months')
plt.ylabel('Frequency')
plt.title('Time of customers in the company')

In [ ]:
surv = lf.WeibullAFTFitter()

surv.fit(df = churn, duration_col = 'AccountAge', event_col = 'Churn', \
         formula = 'C(SubtitlesEnabled) + C(ParentalControl) + MonthlyCharges + C(PaperlessBilling)\
         +C(PaymentMethod) + C(SubscriptionType) + ViewingHoursPerWeek + UserRating + SupportTicketsPerMonth + \
         C(ContentType) + C(MultiDeviceAccess)+ C(DeviceRegistered)+\
            AverageViewingDuration + ContentDownloadsPerMonth + C(GenrePreference) + WatchlistSize')

In [ ]:
surv.print_summary()

In [ ]:
summary = surv.summary

In [ ]:

summary = surv.summary.reset_index()
summary = summary[summary["p"] < 0.05]  
plt.figure(figsize=(10, 12))
sns.barplot(
    data=summary,
    x="coef",
    y="covariate",  
    color="steelblue",
    errorbar=None
)
plt.axvline(0, color="black", linestyle="--")
plt.title("Significent coefs")
plt.xlabel("coef")
plt.ylabel("factor")
plt.tight_layout()
plt.show()


Factors that increase account longevity :
 - Premium Subscription  -> Increases account lifetime by ~17% 
 - Credit Card Payment ->  Associated with ~9% longer account life — possibly indicating more committed users
- Content Type  TV Shows and Movies ->  6–8% longer lifetime
- Content Downloads per Month -> Slight but significant positive effect on retention.

Factors that shorten account lifetime:
- Support Tickets per Month (to ~5%)
- Genre Preferences (Comedy, Sci-Fi, Fantasy, Drama) -> negatively associated with account longevity (up to 12% shorter time)
- Monthly Charges -> Higher charges slightly increase churn risk.
- Watchlist Size: -> A larger watchlist without corresponding viewing activity may indicate disengagement.

To conclude, users with a premium subscription who frequently watch content and use a credit card to pay stay longer. The more problems (tickets) a user has or he prefers comedy/fiction, the higher the risk of churn. Increased activity (views and downloads) lengthens the life span, while expensive pricing and poor user experience shorten it.